# TensorFlow
> - 計算グラフを用いた数値計算を行うためのオープンソースのライブラリ。
> - データの流れをエッジで表した計算グラフによる"Define and Run"
> - TensorFlowでは、「データフローグラフ」の考え方に基づいて処理を作る
     - 機械学習では様々な計算式を用いて処理を行うが、プログラムとして書くからには、一つの計算式を関数として定義し、これを相互に呼び出し合って処理を進めていくことになる
> - CPUやGPUといったデバイス単位だけでなく、ネットワーク上の他のマシンでも実行可能
> - PythonやC++のAPIの他にもJava, Go言語も使用可能
> - 深層学習以外の用途でも使われるように設計されている

**インストール方法**
```bash
pip install tensorflow
```
もし使っているマシンがGPUに対応しているならば、下記のコマンドを使ってインストールしておくと、GPU環境でも簡単にTensorFlowを使うことができる。
```bash
pip install tensorflow-gpu
```

**参考サイト**

 - [TensorFlowのPython APIセクション](https://www.tensorflow.org/api_docs/python/)<br>
 https://www.tensorflow.org/api_docs/python/
 - [TensorFlowのチュートリアル](https://www.tensorflow.org/tutorials)<br>
 https://www.tensorflow.org/tutorials

## TensorFlowの基本的な使い方
TensorFlowでは、「データフローグラフ」の考え方に基づいて処理を作る。そのため、

 1. 変数、placeholder、計算手順等を定義(*Define*)する
 2. データフローグラフを実行(*Run*)する

**"*Define and Run*"**
```python
# Define by Run
z = x * y    # 掛け算した結果の数値がzに格納される

# Define and Run
c = tf.maltiply(a, b)    # 掛け算した結果の実際の数値がcに入るわけではない
# 計算グラフ上のcというシンボリックを別に実行する必要がある
```

 - *Define by Run* : TensorFlow, Theano
 - *Define and Run* : Chainer, Torch

### `tf.Session()` : Seesionオブジェクトを生成する
**<font color="red">※`tf.Session()`を実行すると以下の初期化メソッドが呼ばれる</font>**

```python
sess = tf.Session(
    target='',  # オプション。接続先の実行エンジンを指定する。
    graph=None,  # オプション。使用するデータフローグラフを指定する場合に使用する。
    config=None  # オプション。セッションの構成オプションを指定する。
)
```

**`Sessionオブジェクト.run(opノード)`** : opノード(Tensorオブジェクト)を評価し、計算操作の結果を返す
```python
sess.run(
    fetches,  # 計算グラフのopノード(Tensorオブジェクト)を指定する。
    feed_dict=None,  # 型を指定。指定されていない場合はfetchesから推論された型がセットされる。
    options=None,  # オプション。このオプションを使って特定のステップの振る舞いを制御できる(トレースをオンにするなど)。
    run_metadata=None  # オプション。非テンソルからの出力を収集する。
)
```

In [38]:
import tensorflow as tf

# セッションスタート
sess = tf.Session()

c_tf = tf.constant(10.0, dtype=tf.float32)

sess.run(c_tf)
# セッションクローズ
sess.close()

### `tf.InteractiveSession()` : セッションをインタラクティブモードで実行する
<div style="text-align:center">
    Sessionオブジェクト.run(opノード) → opノード.eval()
</div>

**<font color="red">※InteractiveSessionオブジェクトを生成する</font>**
```python
__init__(
    target='',
    graph=None,
    config=None
)
```

| 引数名 | 説明 |
|:--------|:-------|
| `target` | オプション。接続先の実行エンジンを指定する。 |
| `graph` | オプション。特に使用する計算グラフがある場合に指定する。 |
| `config` | オプション。セッションの構成オプションを指定する。 |

 - 「セッションにテンソルオブジェクトを渡す必要がない」
 - Jupiter Notebookのような対話型(インタラクティブモード)の実行環境において、セッションを気にすることなく、コーディングに集中できるように用意された機能

In [ ]:
import tensorflow as tf

# データフローグラフの作成
rate = tf.constant(0.08, dtype=tf.float32)
price = tf.Variable(1000.0, dtype=tf.float32)
tax = tf.multiply(rate, price)
price_tax = tf.add(tax, price)
update = tf.assign(price, price_tax)

# セッション部
sess = tf.InteractiveSession() # インタラクティブモードでセッション開始
sess.run(tf.global_variables_initializer())
print('rate = ', rate.eval())
print('price = ', price.eval())
update.eval()
print('price(update) = ', price.eval())

sess.close() # セッションを閉じる

### with文でセッション全体をラップする

 - メリット : セッション部とデータフローグラフ作成部を分け、セッションの処理を関数として定義することでわかりやすくする.

```python
with オブジェクトを生成するコード as 変数:
    オブジェクトを用いた処理
```
with以下の処理が終了したら、自動的にcloseされる。

In [ ]:
import tensorflow as tf

# データフローグラフの作成
rate = tf.constant(0.08, dtype=tf.float32)
price = tf.Variable(1000.0, dtype=tf.float32)
tax = tf.multiply(rate, price)
price_tax = tf.add(tax, price)
update = tf.assign(price, price_tax)

# セッションの処理を実行する関数
def run_session():
    sess = tf.Session()    # セッションスタート
    # すべての変数をまとめて初期化
    sess.run(tf.global_variables_initializer())
    print('rate = ', sess.run(rate))
    print('price = ', sess.run(price))
    sess.run(update)    # updateノードを実行
    print('price(update) = ', sess.run(price))
    sess.close()    # セッションクローズ

# セッションを実行
run_session()

In [22]:
import tensorflow as tf

# データフローグラフの作成
rate = tf.constant(0.08, dtype=tf.float32)
price = tf.Variable(1000.0, dtype=tf.float32)
tax = tf.multiply(rate, price)
price_tax = tf.add(tax, price)
update = tf.assign(price, price_tax)

# セッションの処理
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print('rate = ', sess.run(rate))
    print('price = ', sess.run(price))
    sess.run(update)
    print('price(update) = ', sess.run(price))

rate =  0.08
price =  1000.0
price(update) =  1080.0


---
# テンソルを作成する

 - テンソル(tensor) : TensorFlowが計算グラフを操作するために使用する第一のデータ構造

## 固定のテンソル

**すべて0の要素からなるテンソルを作成**
```python
# tf.zeros([行数, 列数])
zero_tsr = tf.zeros([row_dim, col_dim])
```

**すべて1の要素からなるテンソルを作成**
```python
# tf.ones([行数, 列数])
ones_tsr = tf.ones([row_dim, col_dim])
```

**定数で埋められたテンソルを作成**
```python
# tf.fill([行数, 列数], 定数値)
filled_tsr = tf.fill([row_dim, col_dim], 42)
```

**既存の定数からテンソルを作成**
```python
constant_tsr = tf.constant([1, 2, 3])
```
`tf.constant()`を使ってある値を配列にブロードキャストすれば、`tf.fill()`の振る舞いを模倣できる。
```python
tf.fill([row_dim, col_dim], 42)
# ↑のコードは次のように書くことができる.
tf.constant(42, shape=[row_dim, col_dim])
```

## 定数のテンソル : `tf.constant()`
```python
tensorflow.constant(
    value,
    dtype=None,
    shape=None,
    name='Const',
    verify_shape=False
)
```

| 引数 | 説明 |
|:------|:----------------------------------------|
| `value` | 定数値または定数値のリストを指定. 値がリストの場合は、shape引数でサイズ(長さ)を指定する際に、実際のサイズが収まるようにサイズ指定することが必要. リストの長さが指定された要素の数よりも少ない場合は、リストの最後の要素が残りの要素を埋めるために使用される. |
| `dtype` | 型を指定. 指定されていない場合は、valueの値から推論された型がセットされる. |
| `shape` | オプション. データのサイズ(長さ)を指定する |
| `name` | 定数を内部的に識別するための名前. 指定しない場合はデフォルトのConstが使用される |
| `verify_shape` | valueの形状を検証可能にするかどうかを論理値で指定する. デフォルトはFalse(検証可能にしない). |

## 同じような形状のテンソルを複数作成
他のテンソルの形状に基づいて変数を初期化することもできる。
```python
zeros_similar = tf.zeros_like(constant_tsr)
ones_similar = tf.ones_like(constant_tsr)
```
※これらのテンソルは引数のテンソルに依存するため、テンソルを順番に初期化しなければならないことに注意！

## シーケンステンソル
TensorFlowでは、あらかじめ定義された区間を含んだテンソルを指定できる。次の関数の動作は、`range()`の出力やNumPyの`linspace()`の出力と非常によく似ている.

In [5]:
import numpy as np

print([i for i in np.linspace(0, 10, 3)])
print([i for i in range(0, 10, 3)])

[0.0, 5.0, 10.0]
[0, 3, 6, 9]


```python
linear_tsr = tf.linspace(start=0, stop=1, num=3)
>>> [0.0, 0.5, 1.0]
```
```python
integer_seq_tsr = tf.range(start=6, limit=15, delta=3)
>>> [6, 9, 12]
```

In [6]:
print([i for i in np.linspace(0, 1, 3)])
print([i for i in range(6, 15, 3)])

[0.0, 0.5, 1.0]
[6, 9, 12]


## ランダムテンソル
**一様分布から抽出された乱数を生成**
```python
randunif_tsr = tf.random_uniform([row_dim, col_dim], minval=0, maxval=1)
# ( minval <= x < maxval )から抽出される
```

**正規分布から抽出された乱数を生成**
```python
# tf.random_normal([行数, 列数], mean=平均値, stddev=標準偏差)
randnorm_tsr = tf.random_normal([row_dim, col_dim], mean=0.0, stddev=1.0)
```

**指定された平均から標準偏差の2倍までの正規分布から値を抽出**
```python
# tf.truncated_normal([行数, 列数], mean=平均値, stddev=標準偏差)
runcnorm_tsr = tf.truncated_normal([row_dim, col_dim], mean=0.0, stddev=1.0)
```

**配列の要素をランダム化する**
```python
shuffled_output = tf.random_shuffle(input_tensor)
cropped_output = tf.random_crop(input_tensor, crop_size)
```

**RGB画像のサイズのクロップ(切り取り)をランダムに実行できる**
```python
# tf.random_crop(my_image, [高さ, 幅, 3])
cropped_image = tf.random_crop(my_image, [height/2, width/2, 3])
```

## 固定のテンソルから変数テンソルを作成
テンソルの作成方法が決まったら、テンソルを`Variable()`でラップすることで、対応する変数を作成することもできる。
```python
my_var = tf.Variable(tf.zeros([row_dim, col_dim]))
```

## 変数のテンソルを作成 : `tf.Variable()`
> **変数** : アルゴリズムのパラメータであり、TensorFlowはアルゴリズムを最適化するためにそれらのパラメータの変更方法を追跡する

**<font color="red">※`tf.Variable()`を実行すると以下の初期化メソッドが呼ばれる</font>**.
```python
__init__(
    initial_value=None,
    trainable=True,
    collections=None,
    validate_shape=True,
    caching_device=None,
    name='Variable',
    variable_def=None,
    dtype=None,
    expected_shape=None,
    import_scope=None,
    constraint=None
)
```

| 引数名 | 説明 |
| :------- | :------- |
| `initial_value` | 変数の値として使用できるPythonオブジェクト. 初期値を設定しないことも可能だが、その場合は、dtype指定する必要がある.<br> 値がリストの場合は、shape引数でサイズ(長さ)を指定する際に、実際のサイズが収まるようにサイズ指定することが必要. リストの長さが指定された要素の数よりも少ない場合は、リストの最後の要素が残りの要素を埋めるために使用される. |
| `trainable` | デフォルトのTrueで、変数をグラフに追加する. |
| `collections` | グラフコレクションキーのリストで、変数はこれらのコレクションに追加される. デフォルトのNoneで[GraphKeys.GLOBAL_VARIABLES]が使われる. |
| `validate_shape` | Falseに設定すると、変数が未知の値で初期化されるのを許可する. デフォルトのTrueは、initial_valueの値によって形状を決定する |
| `caching_device` | 変数に読み込む値をキャッシュデータとして保存する場合を指定するためのオプション |
| `name` | 変数を内部的に識別するための名前. 指定しない場合はデフォルトのVariableが使用される |
| `expected_shape` | initial_valueの形状をあらかじめ指定する場合に使用する |
| `import_scope` | 変数のスコープとして使用する文字列を設定するためのオプション. プロトコルバッファから初期化するときにのみ使用する |
| `constraint` | 更新(値をセット)された後に変数に適用される関数を指定するオプション. |

In [14]:
sess = tf.Session()

# 変数の設定 = tf.Variable(テンソル)
my_var = tf.Variable(tf.zeros([2, 3]))

# 変数を初期化するノード
initialize_op = tf.global_variables_initializer()
# 変数を初期化
sess.run(initialize_op)

sess.run(my_var)

array([[0., 0., 0.],
       [0., 0., 0.]], dtype=float32)

## placeholder

 - **placeholder** : 特定の型と形状を持つデータを供給できるようにするオブジェクト
 
**placeholder** : 計算グラフに供給するデータの位置を保持しているだけ
 - placeholderは、セッションの `feed_dict` 引数からデータを取得する
 - placeholderを計算グラフに配置するには、そのplaceholderで少なくとも1つの演算を実行しなければならない

定数(Constant)や変数(Variable)は、宣言時に初期値を設定する。ただ、処理によっては実際にプログラムを実行してみないことには値が決まらないこともある。そようなときに使うのが、`tf.placeholder`。

```python
tf.placeholder(
    dtype,
    shape=None,
    name=None
)
```

| 引数名 | 説明 |
|:--------|:------|
| `dtype` | 要素の型 |
| `shape` | オプション。要素の形状を指定。 |
| `name` | オプション。オペレーションの名前。 |

計算グラフを通じてデータを供給する前に、データの形状を設定し、演算の成果指標の形状を知っておく必要がある。だが、常にそうなるとは限らない。事前にはわからない次元が1つか2つあるかもしれないし、それらが変化することも考えられる。そのような場合は、変化する可能性がある、あるいは不明な次元をNoneという値で指定する。

x_data = tf.placeholder(tf.float32, shape=[3, None])

In [11]:
sess = tf.Session()

# tf.placeholder(データ型, shape=[行数, 列数])
x = tf.placeholder(tf.float32, shape=[2, 2])
# 恒等演算(単にxを返す)
y = tf.identity(x)

x_vals = np.random.rand(2, 2)
sess.run(y, feed_dict={x: x_vals})
# sess.run(x, feed_dict={x: x_vals}) が自己参照エラーになることに注意

array([[0.95327884, 0.39202842],
       [0.3763179 , 0.3363032 ]], dtype=float32)

## 変数の初期化
計算グラフを実行するときには、作成した変数をいつ初期化するのかについてTensorFlowに教えてやらなければならない。

 - 変数にはそれぞれ`initializer`メソッドが用意されているが、変数を初期化するための最も一般的な方法は、**ヘルパー関数**(この場合は、`global_variables_initializer()`)を使用することである。
 - この関数は計算グラフに演算を追加するものであり、その計算グラフでは変数がすべて初期化される。

```python
initializer_op = tf.global_variables_initializer()
```

### `Variableオブジェクト.initializer` : 変数の初期化
**変数(Variable)の初期化**
```python
# initializerプロパティの戻り値(初期化op)を取得する.
Sessionオブジェクト.run(Variableオブジェクト.initializer)

# 初期化が済むと他のopノードと同じ状態になる.
Sessionオブジェクト.run(Variableオブジェクト)
```

In [20]:
sess = tf.Session()

first_var = tf.Variable(tf.zeros([2, 3]))
add_first_var = tf.add(first_var, 1)
# 変数first_varは初期化が必要
sess.run(first_var.initializer)
# 変数first_varを出力
print('first_var = ', sess.run(first_var))
# 1を足す
sess.run(add_first_var)
# 更新を確認
print('first_var = ', sess.run(add_first_var))

second_var = tf.Variable(tf.zeros_like(first_var))
# first_varに依存
sess.run(seconde_var.initializer)

# セッションを閉じる
sess.close()

first_var =  [[0. 0. 0.]
 [0. 0. 0.]]
first_var =  [[1. 1. 1.]
 [1. 1. 1.]]


NameError: name 'seconde_var' is not defined

### `tf.global_variables_initializer()`で変数をまとめて初期化する
`tf.global_variables_initializer()`は、定義済みのすべての変数をまとめて初期化する。

> `tf.global_variables_initializer()` : **すべてのグローバル変数を初期化するための処理を返す。**

```python
Sessionオブジェクト.run(tf.global_variables_initializer())
```

In [21]:
# 例)
import tensorflow as tf

# データフローグラフの作成
rate = tf.constant(0.08, dtype=tf.float32)
price = tf.Variable(1000.0, dtype=tf.float32)
tax = tf.multiply(rate, price)
price_tax = tf.add(tax, price)
update = tf.assign(price, price_tax)

# --- セッション部 ---
sess = tf.Session()    # セッションスタート

print('rate = ', sess.run(rate))    # rateノードを実行

sess.run(tf.global_variables_initializer())

print('price = ', sess.run(price))

sess.run(update)    # updateノードを実行
print('price(update) = ', sess.run(price))    # 更新後のpriceを出力

sess.close()    # セッションクローズ
# --- セッション部 ---

rate =  0.08
price =  1000.0
price(update) =  1080.0


# 行列を操作する

In [8]:
import numpy as np
import tensorflow as tf

sess = tf.Session()

## 行列を作成する

In [10]:
identity_matrix = tf.diag([1.0, 1.0, 1.0])    # 1次元の配列やリストから対角行列を作成
A = tf.truncated_normal([2, 3])
B = tf.fill([2, 3], 5.0)
C = tf.random_uniform([3, 2])
D = tf.convert_to_tensor(np.array([[1., 2., 3.], [-3., -7., -1.], [0., 5., -2.]]))

print(sess.run(identity_matrix))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [11]:
print(sess.run(A))

[[ 0.8404328   0.41313443 -0.78452784]
 [ 0.03570103  0.06861581 -1.1504939 ]]


In [12]:
print(sess.run(B))

[[5. 5. 5.]
 [5. 5. 5.]]


In [13]:
print(sess.run(C))

[[0.99865496 0.44942558]
 [0.59569776 0.7610649 ]
 [0.11731505 0.7798513 ]]


In [14]:
print(sess.run(D))

[[ 1.  2.  3.]
 [-3. -7. -1.]
 [ 0.  5. -2.]]


## 行列の加算/減算/乗算

In [15]:
print(sess.run(A + B))

[[5.038511  4.1288133 4.5794473]
 [5.8132734 5.2717886 3.946489 ]]


In [16]:
print(sess.run(B - B))

[[0. 0. 0.]
 [0. 0. 0.]]


### `tf.matmul`
基本的な演算子の次によく使われるものといえば、一般的な行列の掛け算である matmul 演算でしょう。matmul 演算は非常に簡単で、LxN 行列を第一引数に、MxN 行列を第二引数に渡すと、LxM 行列を返してくれます。具体的には、下記の結果を見ていただくのが早いでしょう。

In [17]:
print(sess.run(tf.matmul(B, identity_matrix)))

[[5. 5. 5.]
 [5. 5. 5.]]


In [48]:
# 2×3行列
a = tf.Variable([[1, 2, 3],
                            [3, 4, 5]])
# 3×1行列
y = tf.matmul(a, [[1],
                               [2],
                               [3]])
sess.run(a.initializer)
# 2x3 行列と3x1 行列の掛け算 → 2×1行列
sess.run(y)

array([[14],
       [26]], dtype=int32)

In [53]:
# 3×1行列テンソル
b = tf.Variable([[1], [2], [3]])
o = tf.matmul(a, b)

sess.run(b.initializer)
sess.run(o)

array([[14],
       [26]], dtype=int32)

## 行列の転置

In [18]:
print(sess.run(tf.transpose(C)))

[[0.4336083  0.057863   0.27412355]
 [0.5640781  0.05613565 0.33640802]]


この場合、乱数が再び初期化されるため、以前とは異なる値が得られることに注意！

## 行列式を求める

In [23]:
print(sess.run(tf.matrix_determinant(D)))

-37.99999999999999


## 逆行列を求める

In [24]:
print(sess.run(tf.matrix_inverse(D)))

[[-0.5        -0.5        -0.5       ]
 [ 0.15789474  0.05263158  0.21052632]
 [ 0.39473684  0.13157895  0.02631579]]


## コレスキー分解

In [25]:
print(sess.run(tf.cholesky(identity_matrix)))

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


## 固有値と固有ベクトルを求める

In [26]:
eigenvalues, eigenvectors = sess.run(tf.self_adjoint_eig(D))
print(eigenvalues)
print(eigenvectors)

[-10.65907521  -0.22750691   2.88658212]
[[ 0.21749542  0.63250104 -0.74339638]
 [ 0.84526515  0.2587998   0.46749277]
 [-0.4880805   0.73004459  0.47834331]]


# 演算を設定する

In [27]:
import tensorflow as tf

sess = tf.Session()

## 徐算の計算

In [28]:
print(sess.run(tf.div(3, 4)))
print(sess.run(tf.div(3.0, 4.0)))

0
0.75


浮動小数点数を返す

In [29]:
print(sess.run(tf.truediv(3, 4)))

0.75


浮動小数点数ではあるものの、整数の徐算を行いたい場合は、`floordiv()`という関数を使用する

In [30]:
print(sess.run(tf.floordiv(3.0, 4.0)))

0.0


## 徐算の余りを返す

In [31]:
print(sess.run(tf.mod(22.0, 5.0)))

2.0


## 2つのテンソルの外積を求める
外積が定義されるのは、2つの3要素ベクトルに限られる。

In [32]:
print(sess.run(tf.cross([1.0, 0.0, 0.0], [0.0, 1.0, 0.0])))

[0. 0. 1.]


## 一般的な数学関数

| 演算子 | 説明 |
|:--------|:-------|
| `tf.add(x, y)` | 同じ型のTensorを足し算する($x + y$)。 |
| `tf.abs(x)` | `x`の絶対値を返す |
| `tf.ceil(x)` | `x`の天井関数 |
| `ft.cos(x)` | `x`のcos関数 |
| `tf.subtract(x, y)` | 同じ型のTensorを引き算する($x - y$)。 |
| `tf.sqrt(x)` | $x$の平方根を求める。 |
| `tf.square(x)` | `x`の自乗を計算 |
| `tf.multiply(x, y)` | 2つのTensorに対して行列の積を求める。 |
| `tf.pow(x, y)` | $x$の要素ごとに$y$乗する($x^{y}$)。 |
| `tf.exp(x)` | $\mathrm {exp}\left( x \right)$ |
| `tf.log(x)` | `x`の自然対数を返す |
| `tf.div(x, y)` | $x$と$y$の要素ごとの徐算を行う。 |
| `tf.truediv(x, y)` | float型の引数をキャストする以外は、`div()`と同じ。 |
| `tf.floordiv(x, y)` | 求めた値を整数に丸める以外は、`div()`と同じ。 |
| `tf.mod(x, y)` | 要素ごとの剰余を求める。 |
| `tf.maximum(x, y)` | `x`,`y`の要素単位の最大値 |
| `tf.minimum(x, y)` | `x`, `y`の要素単位の最小値 |

機械学習で使用されていて、TensorFlowにも組み込まれている特別な数学関数

| 関数 | 説明 |
|:------|:------|
| `digamma(x)` | `x`のプサイ関数. `lgamma()`の導関数 |
| `erf(x)` | `x`のガウス誤差関数 |
| `erfc(x)` | `x`の相補誤差関数 |
| `igamma(a, x)` | 第1種不完全ガンマ関数 |
| `igammac(a, x)` | 第2種不完全ガンマ関数 |
| `lbeta(x)` | `x`のベータ関数の絶対値の自然対数 |
| `lgamma(x)` | `x`のガンマ関数の絶対値の自然対数 |
| `squared_difference(x, y)` | `x`と`y`の差の自乗 |

## 自作の関数を作成する
$$
y = 3 x^{2} - x + 10
$$

In [33]:
def custom_polynomial(value):
    return (tf.subtract(3 * tf.square(value), value) + 10)

print(sess.run(custom_polynomial(11)))

362


---
# 入れ子の演算を階層化する
同じ計算グラフに複数の演算を追加する方法について説明する。

<img src="./imgs/02/データが計算グラフをさかのぼっていくときのサイズを確認できる計算グラフ.png" width="30%">

計算グラフの流れ
$$
y = const2 + y1\\
y1 = const1 \times y2\\
y2 = const \times placeholder
$$

In [2]:
import numpy as np
import tensorflow as tf

sess = tf.Session()

In [8]:
my_array = np.array([
        [1.0, 3.0, 5.0, 7.0, 9.0],
        [-2.0, 0.0, 2.0, 4.0, 6.0],
        [-6.0, -3.0, 0.0, 3.0, 6.0]
    ])

# 入力を2つにするために配列を複製
sample_arr = np.array([my_array, my_array + 1])
sample_arr

array([[[ 1.,  3.,  5.,  7.,  9.],
        [-2.,  0.,  2.,  4.,  6.],
        [-6., -3.,  0.,  3.,  6.]],

       [[ 2.,  4.,  6.,  8., 10.],
        [-1.,  1.,  3.,  5.,  7.],
        [-5., -2.,  1.,  4.,  7.]]])

In [4]:
x_ph = tf.placeholder(tf.float32, shape=(3, 5))

const_1 = tf.constant([[1.0], [0.0], [-1.0], [2.0], [4.0]])
const_2 = tf.constant([[2.0]])
a1 = tf.constant([[10.0]])

In [5]:
# 行列の徐算(A[3×5] * m1[5×1] = prod1[3×1])
y2 = tf.matmul(x_ph, const_1)
# 行列の乗算(prod1[3×1] * m2[1×1] = prod2[3×1])
y1 = tf.matmul(y2, const_2)
# 行列の加算(prod2[3×1] + a1[1×1])
y = tf.add(y1, a1)

In [7]:
for x in sample_arr:
    print(sess.run(y, feed_dict={x_ph: x}))

[[102.]
 [ 66.]
 [ 58.]]
[[114.]
 [ 78.]
 [ 70.]]


---
# 複数の層を操作する

In [39]:
import numpy as np
import tensorflow as tf
import os

sess = tf.Session()

## 例
**1. NumPyを使ってサンプルの2D画像を作成する。**これは4×4ピクセルの画像であり、4つの次元で作成する。

In [40]:
x_shape = [1, 4, 4, 1]
x_val = np.random.uniform(size=x_shape)
x_val

array([[[[0.60736108],
         [0.6397809 ],
         [0.19806684],
         [0.46055464]],

        [[0.24045537],
         [0.87856878],
         [0.69671331],
         [0.1944757 ]],

        [[0.55127505],
         [0.69572689],
         [0.6853485 ],
         [0.9049865 ]],

        [[0.56250792],
         [0.41152774],
         [0.02212364],
         [0.57048396]]]])

**2. 次にサンプル画像を供給するためのplaceholderを計算グラフに作成しなければならない**

In [41]:
x_data = tf.placeholder(tf.float32, shape=x_shape)

**3. 4×4画像に対する移動平均ウィンドウを作成するには、2×2の形状のウィンドウに定数を畳み込む関数を使用する**

In [42]:
my_filter = tf.constant(0.25, shape=[2, 2, 1, 1])
my_strides = [1, 2, 2, 1]
mov_avg_layer = tf.nn.conv2d(x_data, my_filter, my_strides, padding="SAME", name='Moving_Avg_Window')

また、`tf.nn.conv2d()`の`name`引数を使って、この層に`Moving_Avg_Window`という名前を付けていることにも注意しよう。

**4. 次に移動平均ウィンドウの2×2出力を操作するカスタム層を定義する**

In [43]:
def custom_layer(input_matrix):
    input_matrix_sqeezed = tf.squeeze(input_matrix)
    A = tf.constant([[1.0, 2.0], [-1.0, 3.0]])
    b = tf.constant(1.0, shape=[2, 2])
    temp1 = tf.matmul(A, input_matrix_sqeezed)
    temp = tf.add(temp1, b)  # Ax + b
    return tf.sigmoid(temp)

**5. 次に、この新しい層を計算グラフに追加する必要がある。次に示すように、この追加は名前付きスコープを使って行うため、新しい層の識別/畳み込み/展開が可能となる。**

In [44]:
with tf.name_scope('Custom_Layer') as scope:
    custom_layer1 = custom_layer(mov_avg_layer)

**6. あとは、placeholderに4×4画像を供給し、TensorFlowに計算グラフの処理を実行させればよい。**

In [45]:
print(sess.run(custom_layer1, feed_dict={x_data: x_val}))

[[0.93714815 0.922651  ]
 [0.8883738  0.90462935]]
